# Nested JSON output 
### Showcase of complex form input handling and structured JSON response
 This pipeline presents a web form capturing user/profile/order/settings data, processes it asynchronously, and returns a nested JSON object.

 ----------------


### Import needed bspump modules:


In [ ]:
from bspump.http.web.server import *
from bspump.jupyter import *
import asyncio

----------------
### In Jupyter, you can test the pipeline logic without submitting the form using a sample event:


In [ ]:
event = {
    "form": {
        "user_name": "Alice",
        "user_email": "alice@example.com",
        "user_age": 30,
        "user_verified": True,
        "newsletter": False,
        "notifications_email": True,
        "notifications_sms": False,
        "order_id": 12345,
        "language": "en",
        "keywords": "alpha,beta,gamma"
    }
}

----------------
### Pipeline Definition: auto_pipeline
Defines the HTTP form source ```WebFormSource``` and JSON sink ```JSONWebSink```.
When deployed, visiting ```http://<host>:<port>/``` shows the form with multiple fields:
user_name, user_email, user_age, user_verified, etc. On submission, the Proscessing block runs and handles the business logic.


In [ ]:
auto_pipeline(
    source=lambda app, pipeline: WebFormSource(
        app, pipeline, route="/", fields=[
            TextField("user_name", display="User Name"),
            TextField("user_email", display="Email"),
            IntField("user_age", display="Age"),
            CheckboxField("user_verified", display="Verified"),
            CheckboxField("newsletter", display="Newsletter Subscription"),
            CheckboxField("notifications_email", display="Email Notifications"),
            CheckboxField("notifications_sms", display="SMS Notifications"),
            IntField("order_id", display="Order ID"),
            TextField("language", display="Language"),
            TextField("keywords", display="Keywords (comma-separated)", multiline=True),
        ]
    ),
    sink=lambda app, pipeline: WebSink(app, pipeline),
    name="WebServerPipeline"
)

----------------
### Processing Step: Pipeline logic with Asynchronous functionality
When the form is submitted, ```bspump``` gathers all form fields into ```event["form"]```.
 The ```asyncio.sleep(1)``` simulates work (sleep), extracts form data,
 builds a nested JSON response, and passes event to the sink.

In [ ]:
print("Processing form data…", event["form"])
# (Optional) Simulate asynchronous work, for example calling an external service
await asyncio.sleep(1)

try:
    form_data = event["form"] # Log raw form values
    # Construction of nested JSON structure from form fields
    event["response"] = json.dumps({
        "user": {
            "name": form_data["user_name"],
            "email": form_data["user_email"],
            "profile": {
                "age": form_data["user_age"],
                "verified": form_data["user_verified"],
                "preferences": {
                    "newsletter": form_data["newsletter"],
                    "notifications": {
                        "email": form_data["notifications_email"],
                        "sms": form_data["notifications_sms"]
                    }
                }
            }
        },
        "order": {
            "id": form_data["order_id"],
            "shipped": False
        },
        "settings": {
            "language": form_data["language"],
            "keywords": form_data["keywords"].split(",")
        },
    })
    event["content_type"] = "text/plain"
# Handle missing form fields
except KeyError as e:
    event["response"] = {"error": f"Missing field: {str(e)}"}
# Handle any other unexpected errors
except Exception as e:
    event["response"] = {"error": f"Unexpected error: {str(e)}"}

----------------
### Testing
You are able to run the notebook yourself by typing:
```bitswan notebook NestedJsonOutput/main.ipynb```
into the console in the examples directory, then you can test it on ```http://localhost:8080/```